# Wrist Movement Data Formating

In [1]:
import numpy as np
import pandas as pd
from math import sqrt,acos,degrees
import matplotlib.pyplot as plt
from scipy.signal import resample
from tqdm.notebook import tqdm
from math import radians, cos, sin
from time import time
import os
import csv
from Data_preparation_Library import*
from openpyxl import load_workbook

## EMG Data Extraction (Cumilative Function)

In [2]:
def extract_dataframes(path,file,save=False,duration=None,dtm_theta=None):
    file_s = file
    file = '/marker_data/' + file
    
    emg_labels = ['Frame','Sub Frame',
                 'IM EMG1',
                 'IM EMG2',
                 'IM EMG3',
                 'IM EMG4',
                 'IM EMG5',
                 'IM EMG6',
                 'IM EMG7',
                 'IM EMG8',
                 'IM EMG9',
                 'IM EMG10',
                 'IM EMG11',
                 'IM EMG12']
    emg_labels_ref = ['Frame','Sub Frame',
                 'EMG1',
                 'EMG2',
                 'EMG3',
                 'EMG4',
                 'EMG5',
                 'EMG6',
                 'EMG7',
                 'EMG8',
                 'EMG9',
                 'EMG10',
                 'EMG11',
                 'EMG12']
    marker_labels = ['Frame','Sub Frame',
                  'RSHO_X','RSHO_Y','RSHO_Z',
                  'RUPA_X','RUPA_Y','RUPA_Z',
                  'RELB_X','RELB_Y','RELB_Z',
                  'RM1_X','RM1_Y','RM1_Z',
                 'RFRM_X','RFRM_Y','RFRM_Z',
                 'WRM2_X','WRM2_Y','WRM2_Z',
                 'RWRA_X','RWRA_Y','RWRA_Z',
                 'RWRB_X','RWRB_Y','RWRB_Z',
                 'RFIN_X','RFIN_Y','RFIN_Z']
    
    pronation_movement = ['No Motion','Supination','Pronation']
    flexion_movement = ['No Motion','Extension','Flexion']
    radial_movement = ['No Motion','Ulnar','Radial']
    dtm_movement = ['No Motion','Forward','Backward']
    
    #################
    # EMG Data Frame#
    #################
    
    typ = 'c'
    if 'Bulb' in file:
        typ = 'b'
        
    emg_df,marker_df = extract_df_xlsx(path+'/'+file,typ)
    emg_df = emg_df[emg_df.columns].astype(float)
    marker_df = marker_df[marker_df.columns].astype(float)
    marker_df = marker_df.interpolate()
    
    # Angles Dataframe
    angles_df = compute_wrist_angles(marker_df,degree=True)
    
    pronations = np.array(angles_df['Pronation_Angle'])
    flexions = np.array(angles_df['Flexion_Angle'])
    radials = np.array(angles_df['Radial_Angle'])
    elbows = np.array(angles_df['Elbow_Joint'])
    
    # Resampling to EMG SR(2000 Hz) from Vicon SR(100 Hz)
    pronations = resample_series(pronations,100,2000)
    flexions = resample_series(flexions,100,2000)
    radials = resample_series(radials,100,2000)
    elbows = resample_series(elbows,100,2000)    
    
    diff_interval = 1000
    
    pronation_labels,pronation_movement_labels = direction_labels(pronations,diff_interval,pronation_movement)
    flexion_labels,flexion_movement_labels = direction_labels(flexions,diff_interval,flexion_movement)
    radial_labels,radial_movement_labels = direction_labels(radials,diff_interval,radial_movement)
    
    # Dart Throwing Motion Labeling
    dtm_labels = []
    ln = len(flexion_labels)
    
    for i in range(ln):
        if(flexion_labels[i]==0 and radial_labels[i]==0):
            dtm_labels.append(0)
        elif(flexion_labels[i]==2 and radial_labels[i]==1):
            dtm_labels.append(1)
        elif(flexion_labels[i]==1 and radial_labels[i]==2):
            dtm_labels.append(2)
        else:
            dtm_labels.append(0)
    
    emg_df['Pronation_Angle'] = pronations
    emg_df['Pronation_Label'] = pronation_labels
    
    emg_df['Flexion_Angle'] = flexions
    emg_df['Flexion_Label'] = flexion_labels
    
    emg_df['Radial_Angle'] = radials
    emg_df['Radial_Label'] = radial_labels
    
    emg_df['Elbow_Joint_Angle'] = elbows
    
    emg_df['DTM_Label'] = dtm_labels

    if dtm_theta != None:
        dtm_angles = get_DTM_angles(marker_df, dtm_theta)
        dtm_angles = resample_series(dtm_angles,100,2000)  
        emg_df['DTM_Angle'] = dtm_angles 
    
    if(save==True):
        emg_df.to_csv(path+'/computed_'+file_s[0:-5]+'.csv')
        
    return emg_df,marker_df, angles_df

In [72]:
def resample_series(data,sr_origin,sr_new):
    """
    Upsamples Series Vector to required Freq(Hz)
    data - Series 1D Array
    sr_origin - Origin Sampling Rate
    sr_new - New Sampling Rate
    Return - Resampled Data to Given Sample Rate
    """
    dt = pd.Series(data)
    dt.index = pd.to_datetime(dt.index*(int((sr_origin/10)**7)))
    dt2 = dt.resample(str(1/sr_new)+'S').ffill()
    len_diff = len(dt)*(sr_new/sr_origin) - len(dt2)
    dt2_list = list(np.array(dt2))
    resampled_array = None
    
    # Balencing
    if(len_diff%2==0):
        nd = int(len_diff/2)
        resampled_array = [dt2_list[0]]*nd + dt2_list
        resampled_array = resampled_array + [dt2_list[-1]]*nd
    else:
        nd = int((len_diff+1)/2)
        resampled_array = [dt2_list[0]]*nd + dt2_list
        resampled_array = resampled_array + [dt2_list[-1]]*nd
        resampled_array = resampled_array[1:]
        
    return np.array(resampled_array)

In [3]:
def resample_series(data,sr_origin,sr_new):
    """
    Upsamples Series Vector to required Freq(Hz)
    data - Series 1D Array
    sr_origin - Origin Sampling Rate
    sr_new - New Sampling Rate
    Return - Resampled Data to Given Sample Rate
    """
    data = np.array(data)
    ln = data.shape[0]
    new_ln = int(ln*(sr_new/sr_origin))
    resampled_array = resample(data,new_ln)    
    return np.array(resampled_array)

## Compute Wrist Angles

In [4]:
def compute_wrist_angles(df,degree=False):
    # Wrist Segment
    WRM2 = df[['WRM2_X','WRM2_Y','WRM2_Z']]
    RWRA = df[['RWRA_X','RWRA_Y','RWRA_Z']]
    RWRB = df[['RWRB_X','RWRB_Y','RWRB_Z']]
    # Palm Segment
    RFIN = df[['RFIN_X','RFIN_Y','RFIN_Z']]

    # Elbow Segment
    RFRM = df[['RFRM_X','RFRM_Y','RFRM_Z']]
    RM1 = df[['RM1_X','RM1_Y','RM1_Z']]
    RELB = df[['RELB_X','RELB_Y','RELB_Z']]
    # Shoulder Segment
    RSHO = df[['RSHO_X','RSHO_Y','RSHO_Z']]
    RUPA = df[['RUPA_X','RUPA_Y','RUPA_Z']]
    # Bisector Point
    MID = (np.array(RWRB) + np.array(RWRA))/2
    MIDE = (np.array(RFRM) + np.array(RM1))/2

    # Translate Wrist to Elbow Segment Mid
    RWRB_E = RWRB - MIDE

    flexion_angles = angles_lines(RFIN,WRM2,MID,deg=degree)
    radial_angles = angles_lines(RFIN,RWRB,MID,deg=degree)-90
    pronation_angles = angles_lines(RFRM,RWRB_E,MIDE,deg=degree)
    elbow_angles = angles_lines(RSHO,MID,MIDE,deg=degree)
    
    df_labels = ['Flexion_Angle','Radial_Angle','Pronation_Angle','Elbow_Joint']
#     df_labels = ['Pitch','Yaw','Roll','Elbow_Joint']
    ndf = pd.DataFrame(columns=df_labels)
    ndf['Flexion_Angle'] = flexion_angles
    ndf['Radial_Angle'] = radial_angles
    ndf['Pronation_Angle'] = pronation_angles
    ndf['Elbow_Joint'] = elbow_angles
    return ndf

def angles_lines(p1,p2,mid,deg=False):
    u = np.array(p1)-np.array(mid)
    v = np.array(p2)-np.array(mid)
    i1,j1,k1 = u[:,0],u[:,1],u[:,2]
    i2,j2,k2 = v[:,0],v[:,1],v[:,2]
    angles = []
    for t in range(len(i1)):
        cos_t = abs(i1[t]*i2[t]+j1[t]*j2[t]+k1[t]*k2[t])
        cos_t = cos_t/(sqrt(i1[t]**2+j1[t]**2+k1[t]**2)*sqrt(i2[t]**2+j2[t]**2+k2[t]**2))
        if deg==False:
            angles.append(acos(cos_t))  
        if deg==True:
            angles.append(degrees(acos(cos_t)))
    return np.array(angles)

## Movement Labelling

In [5]:
def direction_labels(array,interval=50,movements=None,angle_thresh=5):
    """
    0 - No Motion
    1 - Positive Direction
    2 - Negative Direction
    """
    labels = []
    diff_arr = difference(array,interval)
    data_len = len(array)
    len_diff = int(data_len - len(diff_arr))
    
    # Hot Labelling
    for diff in diff_arr:
        if(abs(diff)>angle_thresh):
            labels.append(0)
        elif(diff>0):
            labels.append(1)
        elif(diff<0):
            labels.append(2)
            
    # Balencing
    if(len_diff%2==0):
        nd = int(len_diff/2)
        labels = [labels[0]]*nd + labels
        labels = labels + [labels[-1]]*nd
    else:
        nd = int((len_diff+1)/2)
        labels = [labels[0]]*nd + labels
        labels = labels + [labels[-1]]*nd 
        labels = labels[1:]
        
    # Movement Labelling
    if(movements==None):
        return np.array(labels)
    else:
        movement_labels = []
        hot_labels = labels
        for lb in labels:
            movement_labels.append(movements[lb])
        return hot_labels,movement_labels

In [6]:
def direction_labels(array,interval=50,movements=None,angle_thresh=3):
    """
    0 - No Motion
    1 - Positive Direction
    2 - Negative Direction
    """
    labels = []
    data = np.array(array)
    avg = (data.max()-data.min())/2
    for x in list(data):
        if(abs(avg-x)<angle_thresh):
            labels.append(0)
        else:
            labels.append(3)
            
    n = len(array)/2000
    split_list = split(data,int(n))
    group_labels = []
    for arr in split_list:
        if(difference(arr).mean()>0):
            group_labels.append(1)
        else:
            group_labels.append(2)
    direction_labels = []        
    for gl in group_labels:
        for i in range(2000):
            direction_labels.append(gl)
    for i in range(len(direction_labels)):
        if(labels[i]!=0):
            labels[i]=direction_labels[i]
            
    # Movement Labelling
    if(movements==None):
        return np.array(labels)
    else:
        movement_labels = []
        hot_labels = labels
        for lb in labels:
            movement_labels.append(movements[lb])
        return hot_labels,movement_labels

def split(a, n):
    k, m = divmod(len(a), n)
    return list((a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)))

def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return np.array(diff)


In [7]:
def direction_labels(array,interval=50,movements=None):
    """
    0 - No Motion
    1 - Positive Direction
    2 - Negative Direction
    """
    labels = [0]*interval
    i=interval
    std = np.array(array).std()
    while(len(labels)<len(array)):
        diff = difference(array[i-interval:i]).mean()
        if(abs(diff)<std/1000):
            labels.append(0)
        elif(diff>0):
            labels.append(1)
        elif(diff<0):
            labels.append(2)
        i=i+1
    # Movement Labelling
    if(movements==None):
        return np.array(labels)
    else:
        movement_labels = []
        hot_labels = labels
        for lb in labels:
            movement_labels.append(movements[lb])
        return hot_labels,movement_labels

def difference(dataset, interval=1):
	diff = []
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return np.array(diff)


In [8]:
def emg_xlsx(workbook,typ):
    emg_labels = ['Frame','Sub Frame',
                 'EMG1',
                 'EMG2',
                 'EMG3',
                 'EMG4',
                 'EMG5',
                 'EMG6',
                 'EMG7',
                 'EMG8',
                 'EMG9',
                 'EMG10',
                 'EMG11',
                 'EMG12']
    
    sheet = workbook.active
    if(typ=='b'):
        table = sheet['BO6':'BZ92005']
        frames = sheet['A6':'B92005']
    else:
        table = sheet['BO6':'BZ54005']
        frames = sheet['A6':'B54005']
    table = tuple(table)
    frames = tuple(frames)
    frames_arr = []
    emg_arr = []
    for t in range(len(table)):
        row = []
        for i in range(12):
            row.append(table[t][i].value)
        frame = []
        for i in range(2):
            frame.append(frames[t][i].value)
        emg_arr.append(row)
        frames_arr.append(frame)

    emg_arr = np.array(emg_arr)
    frames_arr = np.array(frames_arr)

    emg_df = pd.DataFrame(columns=emg_labels)

    for i in range(2):
        emg_df[emg_labels[i]] = frames_arr[:,i]
    emg_labels = emg_labels[2:]
    for i in range(12):
        emg_df[emg_labels[i]] = emg_arr[:,i]
    return emg_df

def marker_xlsx(workbook,typ):
    marker_labels = ['Frame','Sub Frame',
                  'RSHO_X','RSHO_Y','RSHO_Z',
                  'RUPA_X','RUPA_Y','RUPA_Z',
                  'RELB_X','RELB_Y','RELB_Z',
                  'RM1_X','RM1_Y','RM1_Z',
                 'RFRM_X','RFRM_Y','RFRM_Z',
                 'WRM2_X','WRM2_Y','WRM2_Z',
                 'RWRA_X','RWRA_Y','RWRA_Z',
                 'RWRB_X','RWRB_Y','RWRB_Z',
                 'RFIN_X','RFIN_Y','RFIN_Z']
    sheet = workbook.active
    if(typ=='b'):
        table = sheet['A92012':'AC96611']
    else:
        table = sheet['A54012':'AC56711']
    table = tuple(table)
    emg_arr = []
    for t in range(len(table)):
        row = []
        for i in range(29):
            row.append(table[t][i].value)
        emg_arr.append(row)
    emg_arr = np.array(emg_arr)
    marker_df = pd.DataFrame(columns=marker_labels)
    for i in range(29):
        marker_df[marker_labels[i]] = emg_arr[:,i]
    return marker_df

def extract_df_xlsx(filename,typ):
    workbook = load_workbook(filename)
    emg_df = emg_xlsx(workbook,typ)
    marker_df = marker_xlsx(workbook,typ)
    return emg_df, marker_df

## DTM Angles Calculation

In [9]:
def get_RDTM(A,B,D,theta):
    Ax,Ay,Az = A[0],A[1],A[2]
    Bx,By,Bz = B[0],B[1],B[2]
    Dx,Dy,Dz = D[0],D[1],D[2]

    a,b,c = plane_normal(A,B,D)

    v = normalize_vector([Dx - Bx,Dy - By,Dz - Bz])
    lv,mv,nv = v

    m = (cos(theta) - sin(theta)*b*lv/nv - (sin(theta)*a*lv**2)/nv*mv - sin(theta)*a*nv/mv)
    m = m / ((lv**2 + mv**3 + nv**2)/mv)
    l = sin(theta)*b/nv + lv*sin(theta)*a/(nv*mv) + lv*m/mv
    n = (sin(theta)*a + nv*m)/mv
    
    y = normalize_vector([l,m,n])
    l,m,n = v
    
    return intersection_lines([l,m,n],B,[lv,mv,nv],A)

def plane_normal(P1,P2,P3):
    P12 = np.array(P2) - np.array(P1)
    P13 = np.array(P3) - np.array(P1)
    normal = np.cross(P12,P13)
    normal = normalize_vector(normal)
    return normal

def intersection_lines(v,B,y,A):
    l,m,n = v
    lv,mv,lv = y
    Bx, By, Bz = [B[0], B[1], B[2]]
    Ax, Ay, Az = [A[0], A[1], A[2]]
    
    lam = lv*(By-Ay) - mv*(Bx+Ax)
    lam = lam/(l*mv - m*lv)
    mu = (Bx - Ax + lam*l)/lv
    
    x = Bx + lam*l
    y = By + lam*m
    z = Bz + lam*n
    
    return np.array([x,y,z])

def normalize_vector(v):
    v = np.array(v)
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    else:
        return v / norm

def get_DTM_angles(marker_df, theta):
    # Main Function to Get DTM Angles from Marker DF
    all_RDTM = get_RDTM_df(marker_df,theta)
    
    RFIN = marker_df[['RFIN_X', 'RFIN_Y','RFIN_Z']].values
    WRM2 = marker_df[['WRM2_X', 'WRM2_Y','WRM2_Z']].values
    RWRA = marker_df[['RWRA_X','RWRA_Y', 'RWRA_Z']].values
    RWRB = marker_df[['RWRB_X', 'RWRB_Y','RWRB_Z']].values
    MID = (np.array(RWRB) + np.array(RWRA))/2
    
    dtm_anles = angles_lines(all_RDTM,WRM2,MID,deg=True)
    # Angle Filter Parameters
    dtm_anles_fl = filter_array(dtm_anles,order=4,cf=5,fs=100)
    return dtm_anles_fl

def get_RDTM_df(marker_df, theta):
    RFIN = marker_df[['RFIN_X', 'RFIN_Y','RFIN_Z']].values
    WRM2 = marker_df[['WRM2_X', 'WRM2_Y','WRM2_Z']].values
    RWRA = marker_df[['RWRA_X','RWRA_Y', 'RWRA_Z']].values
    RWRB = marker_df[['RWRB_X', 'RWRB_Y','RWRB_Z']].values
    MID = (np.array(RWRB) + np.array(RWRA))/2
    theta = radians(30) # degrees

    A_all= RFIN
    B_all= MID
    D_all= RWRB

    RDTM_all = []
    for i in range(marker_df.shape[0]):
        A = A_all[i]
        B = B_all[i]
        D = D_all[i]
        x,y,z = get_RDTM(A,B,D,theta)
        rdtm = [x,y,z]
        RDTM_all.append(rdtm)

    return np.array(RDTM_all)

## Data Generation Section

In [15]:
files = []
# trial_names = ['Bulb','Hammer','Cup','Screw']
trial_names = ['Hammer']
for exp in trial_names:
    for i in range(3):
        if(i==0):
            files.append(exp+'.xlsx')
        else:
            files.append(exp+'0'+str(i)+'.xlsx')

path = 'Subjects/Nirmayee'
files

['Hammer.xlsx', 'Hammer01.xlsx', 'Hammer02.xlsx']

In [16]:
start = time()
for file in tqdm(files):
    extract_dataframes(path,file,save=True,dtm_theta=90+0.53)
    print('Eapsed-',time()-start,'s')
    start = time()

Eapsed- 188.22549772262573 s


<ipython-input-9-451d59b503e5>:11: RuntimeWarning: divide by zero encountered in double_scalars
  m = (cos(theta) - sin(theta)*b*lv/nv - (sin(theta)*a*lv**2)/nv*mv - sin(theta)*a*nv/mv)
<ipython-input-9-451d59b503e5>:11: RuntimeWarning: invalid value encountered in double_scalars
  m = (cos(theta) - sin(theta)*b*lv/nv - (sin(theta)*a*lv**2)/nv*mv - sin(theta)*a*nv/mv)
<ipython-input-9-451d59b503e5>:13: RuntimeWarning: divide by zero encountered in double_scalars
  l = sin(theta)*b/nv + lv*sin(theta)*a/(nv*mv) + lv*m/mv
<ipython-input-9-451d59b503e5>:13: RuntimeWarning: invalid value encountered in double_scalars
  l = sin(theta)*b/nv + lv*sin(theta)*a/(nv*mv) + lv*m/mv
<ipython-input-9-451d59b503e5>:36: RuntimeWarning: divide by zero encountered in double_scalars
  mu = (Bx - Ax + lam*l)/lv


Eapsed- 198.16182208061218 s


<ipython-input-9-451d59b503e5>:12: RuntimeWarning: divide by zero encountered in double_scalars
  m = m / ((lv**2 + mv**3 + nv**2)/mv)
<ipython-input-9-451d59b503e5>:12: RuntimeWarning: invalid value encountered in double_scalars
  m = m / ((lv**2 + mv**3 + nv**2)/mv)
<ipython-input-9-451d59b503e5>:35: RuntimeWarning: divide by zero encountered in double_scalars
  lam = lam/(l*mv - m*lv)
<ipython-input-9-451d59b503e5>:39: RuntimeWarning: invalid value encountered in double_scalars
  y = By + lam*m


Eapsed- 193.5937693119049 s



In [17]:
files = []
# trial_names = ['Bulb','Hammer','Cup','Screw']
trial_names = ['Cup']
for exp in trial_names:
    for i in range(3):
        if(i==0):
            files.append(exp+'.xlsx')
        else:
            files.append(exp+'0'+str(i)+'.xlsx')

files

['Cup.xlsx', 'Cup01.xlsx', 'Cup02.xlsx']

In [18]:
start = time()
for file in tqdm(files):
    extract_dataframes(path,file,save=True,dtm_theta=90-6.68)
    print('Eapsed-',time()-start,'s')
    start = time()

Eapsed- 189.59138894081116 s
Eapsed- 189.9456171989441 s


<ipython-input-9-451d59b503e5>:11: RuntimeWarning: divide by zero encountered in double_scalars
  m = (cos(theta) - sin(theta)*b*lv/nv - (sin(theta)*a*lv**2)/nv*mv - sin(theta)*a*nv/mv)
<ipython-input-9-451d59b503e5>:11: RuntimeWarning: invalid value encountered in double_scalars
  m = (cos(theta) - sin(theta)*b*lv/nv - (sin(theta)*a*lv**2)/nv*mv - sin(theta)*a*nv/mv)
<ipython-input-9-451d59b503e5>:13: RuntimeWarning: divide by zero encountered in double_scalars
  l = sin(theta)*b/nv + lv*sin(theta)*a/(nv*mv) + lv*m/mv
<ipython-input-9-451d59b503e5>:13: RuntimeWarning: invalid value encountered in double_scalars
  l = sin(theta)*b/nv + lv*sin(theta)*a/(nv*mv) + lv*m/mv
<ipython-input-9-451d59b503e5>:36: RuntimeWarning: divide by zero encountered in double_scalars
  mu = (Bx - Ax + lam*l)/lv


Eapsed- 197.32296919822693 s



## Batch Generation

In [14]:
subjects = ['Fazil','Ingy','Varun','Abhishek','Nikhil','Jai']
for subject in subjects:
    print('Generating '+subject+' Data')
    path = 'Subjects/' + subject
    start = time()
    for file in tqdm(files):
        extract_dataframes(path,file,save=True)
        start = time()
    print('Eapsed-',time()-start,'s')

Generating Fazil Data



Eapsed- 0.0019953250885009766 s
Generating Ingy Data



Eapsed- 0.0 s
Generating Varun Data



Eapsed- 0.002956867218017578 s
Generating Abhishek Data



Eapsed- 0.001995086669921875 s
Generating Nikhil Data



Eapsed- 0.001995086669921875 s
Generating Jai Data


ValueError: could not convert string to float: ''

## Testing Section

In [111]:
file = 'Bulb.csv'
emg_df,marker_df, angles_df = extract_dataframes(path,file,save=False)

In [112]:
array = np.array(emg_df['Flexion_Angle'])
labels = direction_labels(array)

In [113]:
emg_df[['Pronation_Angle','Flexion_Angle','Radial_Angle','Elbow_Joint_Angle']].describe()

,Pronation_Angle,Flexion_Angle,Radial_Angle,Elbow_Joint_Angle
count,92000.000000,92000.000000,92000.000000,92000.000000
mean,69.098774,69.937462,-16.035652,40.584059
std,9.580746,12.257754,5.708734,5.116913
min,27.789201,32.985233,-26.021371,24.677451
25%,69.037148,66.180474,-20.958741,38.218689
50%,71.405735,70.698310,-17.118221,41.590666
75%,73.773633,78.378348,-11.223833,44.033964
max,89.915761,90.104144,-2.636712,50.846596


In [114]:
emg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92000 entries, 0 to 91999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Frame              92000 non-null  float64
 1   Sub Frame          92000 non-null  float64
 2   EMG1               92000 non-null  float64
 3   EMG2               92000 non-null  float64
 4   EMG3               92000 non-null  float64
 5   EMG4               92000 non-null  float64
 6   EMG5               92000 non-null  float64
 7   EMG6               92000 non-null  float64
 8   EMG7               92000 non-null  float64
 9   EMG8               92000 non-null  float64
 10  EMG9               92000 non-null  float64
 11  EMG10              92000 non-null  float64
 12  EMG11              92000 non-null  float64
 13  EMG12              92000 non-null  float64
 14  Pronation_Angle    92000 non-null  float64
 15  Pronation_Label    92000 non-null  int64  
 16  Flexion_Angle      920

In [115]:
marker_df

,Frame,Sub Frame,RSHO_X,RSHO_Y,RSHO_Z,RUPA_X,RUPA_Y,RUPA_Z,RELB_X,RELB_Y,...,WRM2_Z,RWRA_X,RWRA_Y,RWRA_Z,RWRB_X,RWRB_Y,RWRB_Z,RFIN_X,RFIN_Y,RFIN_Z
0,1.0,0.0,-96.6457,1308.76,1536.45,-244.132,1314.85,1498.23,-352.630,1327.47,...,1472.45,-617.538,1226.61,1457.00,-622.412,1295.58,1448.14,-716.119,1263.08,1460.92
1,2.0,0.0,-96.6258,1308.74,1536.48,-244.105,1314.82,1498.27,-352.597,1327.44,...,1472.52,-617.539,1227.10,1456.74,-622.371,1295.53,1448.22,-716.074,1263.02,1461.02
2,3.0,0.0,-96.5514,1308.70,1536.46,-244.069,1314.77,1498.30,-352.559,1327.38,...,1472.60,-617.503,1227.04,1456.84,-622.329,1295.48,1448.30,-716.061,1262.96,1461.15
3,4.0,0.0,-96.5340,1308.65,1536.48,-244.037,1314.75,1498.35,-352.526,1327.36,...,1472.68,-617.484,1226.98,1456.90,-622.309,1295.43,1448.39,-716.030,1262.89,1461.26
4,5.0,0.0,-96.5155,1308.62,1536.50,-244.037,1314.72,1498.39,-352.523,1327.31,...,1472.78,-617.686,1227.51,1456.83,-622.300,1295.36,1448.50,-716.014,1262.82,1461.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,4596.0,0.0,-92.4231,1316.12,1539.91,-234.969,1335.21,1524.95,-349.448,1360.83,...,1539.28,-618.993,1287.26,1523.76,-616.376,1356.24,1519.81,-714.296,1332.07,1536.25
4596,4597.0,0.0,-92.5227,1315.92,1539.86,-235.128,1334.96,1524.88,-349.555,1360.82,...,1538.92,-619.080,1287.14,1523.25,-616.484,1356.14,1519.58,-714.416,1331.92,1535.77
4597,4598.0,0.0,-92.3805,1315.64,1539.73,-235.243,1334.62,1524.84,-349.635,1360.90,...,1538.60,-619.172,1287.03,1522.82,-616.582,1356.03,1519.35,-714.561,1331.79,1535.40
4598,4599.0,0.0,-92.5342,1315.37,1539.74,-235.346,1334.31,1524.79,-349.753,1361.42,...,1538.29,-619.302,1286.90,1522.44,-616.667,1355.93,1519.12,-714.689,1331.63,1535.01


In [116]:
angles_df

,Flexion_Angle,Radial_Angle,Pronation_Angle,Elbow_Joint
0,43.530307,-4.530843,42.493675,28.868488
1,43.782473,-4.369982,42.505864,28.851117
2,43.791571,-4.356800,42.529980,28.830882
3,43.832238,-4.346455,42.507890,28.840650
4,43.765123,-4.033272,42.464059,28.824247
...,...,...,...,...
4595,41.427857,-3.392353,38.390226,28.050419
4596,41.380162,-3.424816,38.408012,27.980692
4597,41.380016,-3.441472,37.871316,28.036360
4598,41.249275,-3.399838,37.980724,27.984827


In [2]:
from openpyxl import load_workbook

In [8]:
workbook = load_workbook(filename="Subjects/Nirmayee/marker_data/Bulb.xlsx")

In [9]:
workbook.sheetnames

['Bulb']

In [10]:
sheet = workbook.active

In [11]:
sheet['BZ6'].value

-8.74e-06

In [39]:
len()

12

In [122]:
table = sheet['BO6':'BZ92005']
frames = sheet['A6':'B92005']

In [85]:
table = tuple(table)

In [8]:
start = time()
filename="Subjects/Nirmayee/marker_data/Bulb.xlsx"
emg_df, marker_df = extract_df_xlsx(filename,'b')
print(start - time())

-219.78036999702454


In [9]:
emg_df

,Frame,Sub Frame,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12
0,1,0,-3.830000e-06,-0.000039,-0.000046,0.000009,-0.000020,0.000022,-0.000029,-0.000012,-0.000026,-0.000003,-0.000059,-0.000009
1,1,1,-2.040000e-06,-0.000040,-0.000046,0.000009,-0.000020,0.000023,-0.000038,-0.000011,-0.000024,-0.000004,-0.000057,-0.000007
2,1,2,-5.850000e-07,-0.000040,-0.000045,0.000009,-0.000021,0.000024,-0.000044,-0.000010,-0.000023,-0.000006,-0.000057,-0.000005
3,1,3,1.710000e-07,-0.000038,-0.000043,0.000009,-0.000023,0.000023,-0.000043,-0.000012,-0.000023,-0.000007,-0.000058,-0.000003
4,1,4,4.460000e-07,-0.000034,-0.000042,0.000009,-0.000024,0.000022,-0.000035,-0.000016,-0.000024,-0.000009,-0.000060,-0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91995,4600,15,1.410000e-05,-0.000070,-0.000043,0.000018,-0.000021,0.000023,-0.000023,0.000022,-0.000034,0.000005,-0.000061,-0.000012
91996,4600,16,7.750000e-06,-0.000069,-0.000044,0.000013,-0.000021,0.000024,-0.000024,0.000021,-0.000034,0.000004,-0.000061,-0.000015
91997,4600,17,-9.310000e-07,-0.000072,-0.000046,0.000008,-0.000022,0.000023,-0.000024,0.000019,-0.000033,0.000004,-0.000061,-0.000015
91998,4600,18,-1.250000e-05,-0.000080,-0.000046,0.000009,-0.000022,0.000022,-0.000025,0.000016,-0.000033,0.000005,-0.000061,-0.000014


In [10]:
marker_df

,Frame,Sub Frame,RSHO_X,RSHO_Y,RSHO_Z,RUPA_X,RUPA_Y,RUPA_Z,RELB_X,RELB_Y,...,WRM2_Z,RWRA_X,RWRA_Y,RWRA_Z,RWRB_X,RWRB_Y,RWRB_Z,RFIN_X,RFIN_Y,RFIN_Z
0,1,0,-16.6009,-1469.97,1383.3,93.1025,-1473.84,1327.2,205.589,-1457.37,...,1346.17,483.935,-1420.53,1339.12,477.841,-1470.03,1339.85,553.114,-1450.25,1360.59
1,2,0,-16.5531,-1469.91,1383.31,93.1427,-1473.81,1327.23,205.606,-1457.38,...,1346.05,483.907,-1420.51,1338.95,477.89,-1470.05,1339.8,553.148,-1450.25,1360.44
2,3,0,-16.5098,-1469.85,1383.33,93.1797,-1473.77,1327.22,205.632,-1457.39,...,1345.96,484.061,-1420.54,1338.93,477.935,-1470.05,1339.69,553.218,-1450.29,1360.33
3,4,0,-16.4448,-1469.81,1383.36,93.2376,-1473.74,1327.22,205.684,-1457.39,...,1345.85,484.066,-1420.54,1338.92,477.979,-1470.1,1339.66,553.261,-1450.35,1360.29
4,5,0,-16.4011,-1469.76,1383.36,93.2769,-1473.71,1327.21,205.745,-1457.39,...,1345.85,484.125,-1420.64,1338.87,478.036,-1470.13,1339.62,553.292,-1450.43,1360.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,4596,0,-18.2439,-1476.28,1383.46,93.2481,-1477.19,1329.97,206.626,-1459.05,...,1357.32,482.217,-1403.96,1350.77,477.503,-1453.97,1351.46,557.452,-1433.41,1361.07
4596,4597,0,-18.1065,-1476.35,1383.43,93.3627,-1477.22,1329.85,206.744,-1459.07,...,1357.02,482.232,-1403.72,1350.48,477.695,-1453.71,1351.17,557.496,-1433.05,1360.91
4597,4598,0,-17.9564,-1476.44,1383.39,93.5142,-1477.29,1329.71,206.855,-1459.09,...,1356.74,482.324,-1403.47,1350.19,477.787,-1453.48,1350.88,557.597,-1432.67,1360.66
4598,4599,0,-17.7869,-1476.5,1383.35,93.6662,-1477.29,1329.6,206.971,-1459.12,...,1356.44,482.425,-1403.23,1349.87,477.944,-1453.22,1350.6,557.674,-1432.31,1360.42


In [68]:
S = 'gtakl.xlsx'

In [69]:
S[0:-5]

'gtakl'